### Importing libraries

In [ ]:
from geopy.distance import geodesic
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import googlemaps
import random

### Getting the google api key

In [ ]:
gmaps_key = googlemaps.Client(key = '')

### Reading the csv file with the predefined location names of locations in kolkata

In [ ]:
#DATASET
file = 'locations.csv'
df = pd.read_csv(file)
df.head()

### Getting the latitudes and longitudes of the location names and saving the location names along with their location coordinates in a separate csv file

In [ ]:
for i in range(len(df)):
    result = gmaps_key.geocode(df.iloc[i,0] + ' ,kolkata')
    try:
        lat = result[0]['geometry']['location']['lat']
        lon = result[0]['geometry']['location']['lng']
        df['lat'][i] = lat
        df['lng'][i] = lon

    except:
        lat = None
        lon = None
        
        
# df.to_csv('location_final.csv')

### Reading the csv file with locations and coordinates

In [ ]:
data = pd.read_csv('location_final.csv')
print(len(data))
data.head(10)

# Clustering

#### Getting the list of tuples containing the location coordinates of the places

In [ ]:
C = []
for i in range(len(data)):
    C.append((data['lat'][i], data['lng'][i]))

In [ ]:
X = data.loc[:,['name','lat','lng']]
X.head(10)

### Testing to see what number of clusters will provide best results.

#### Calling the KMeans classifier and fitting it into the coordinates

In [ ]:
C = []
for i in range(len(data)):
    C.append((data['lat'][i], data['lng'][i]))
kmeans = [KMeans(n_clusters = i) for i in range(1,15)]
score = [kmeans[i].fit(C).score(C) for i in range(len(kmeans))]
plt.plot(range(1,15), score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

#### Plotting score for each number of clusters

In [ ]:
plt.plot(range(1,15), score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

#### Getting our desired cluster number and using it to build a clustering instance and fitting it on the coordinates

In [ ]:
no = 2
kmeans_final = KMeans(n_clusters = no, random_state = 0).fit(C)

#### Predicting the clusters in which the location falls and adding them to our dataframe

In [ ]:
X['cluster_label'] = kmeans_final.fit_predict(C)
centers = kmeans_final.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans_final.predict(C) # Labels of each point
X.head(5)

#### Visualising our clustered locations

In [ ]:
X.plot.scatter(x = 'lat', y = 'lng', c=labels, s=1, cmap = 'inferno')
plt.legend()
# plt.scatter(centers[:, 0], centers[:, 1], c='black', s=50, alpha=0.5)

#### Saving our labeled dataframe into a final csv file

In [ ]:
# X.to_csv('Location_labeled.csv')

In [ ]:
#cluster_labels gets the labels
cluster_labels = kmeans_final.labels_

# label_arr will contain all locations which fall under a cluster 
# i.e label_arr[0] will give all the locations falling under cluster 0 
label_arr = {alpha : [] for alpha in range(no)}                                                
for i in range(len(cluster_labels)):
    label_arr[cluster_labels[i]].append(i)

In [ ]:
# x will contain the names of all the places which fall in the given cluster
cluster = 0
x = [X['name'][label_arr[cluster][i]] for i in range(len(label_arr[cluster]))]
print(x)

#### Getting the names, coordinates and label of each location in the variables

In [ ]:
names = [X['name'][i] for i in range(len(X))]
coordinates = [(X['lat'][i], X['lng'][i]) for i in range(len(X))]
areas = [X['cluster_label'][i] for i in range(len(X))]

# Booking a Taxi

#### Defining a Taxi object

In [ ]:
class Taxi:
    
    def __init__(self,key,index,number):
        self.key = key
        self.location_coor = coordinates[index]
        self.location_name = names[index]
        self.area = areas[index]
        self.number = number

#### Function to make taxies and allocate them random locations

In [ ]:
def make_taxis(no_taxis):
    number_list = [random.randint(2013,8013) for i in range(no_taxis)]
    index_list = [random.randint(0,len(X)-1) for i in range(no_taxis)]
    taxis = [Taxi(i,index_list[i],number_list[i]) for i in range(no_taxis)]
    return taxis

#### Function to ask for the users location

In [ ]:
def ask_location(location):
    r = gmaps_key.geocode(location + ' ,kolkata')
    try:
        lat = r[0]['geometry']['location']['lat']
        lon = r[0]['geometry']['location']['lng']
    except:
        lat = None
        lon = None

    if(lat!=None):
        arr = [(lat,lon)]
        label = kmeans_final.predict(arr)

        
        
    return arr[0],label[0]

#### Function to allocate a taxi to the user

In [ ]:
def allocate_taxi(taxis,location_coordinates,label):
    
    minimum = 100
    taxi = None
    taxis_in_area = []
    for i in taxis:
        if(i.area == label):
            taxis_in_area.append(i)
            
    for i in taxis_in_area:
        dist = compute_distance(location_coordinates, i.location_coor)
        if(dist<minimum):
            minimum = dist
            taxi = i
        else:
            continue
            
    return taxi, taxis_in_area


#### Function to compute the euclidean distance between two locations

In [ ]:
def compute_distance(x,y):
    return (gmaps_key.distance_matrix(x,y)['rows'][0]['elements'][0]['distance']['value']/1000)

In [ ]:
def compute_time(x,y):
    time_s = gmaps_key.distance_matrix(x,y)['rows'][0]['elements'][0]['duration']['value']
    sec = time_s%60
    time_m = int(time_s/60)
    return time_m

In [ ]:
def compute_fare(x,y):
    
    now = datetime.now()
    current_time = now.hour
    if(current_time<18 and current_time>6):
        rate = 13
    else:
        rate = 13*1.5
    
    dist = (float)(compute_distance(x,y))
    time_required = compute_time(x,y)
    if (dist<=2):
        fare = 25 + time_required*0.2
    else:
        fare = 25 + (dist-2)*rate + time_required*0.2
        
    return fare

In [ ]:
def get_coor(location):
    result = gmaps_key.geocode(drop_loc + ' ,kolkata')
    try:
        lat = result[0]['geometry']['location']['lat']
        lon = result[0]['geometry']['location']['lng']

    except:
        lat = None
        lon = None
        
    return (lat,lon)

In [ ]:
no_taxis = 100
taxis = make_taxis(no_taxis)
for i in taxis:
    print(f"Key : {i.key}, Location : {i.location_name}, coordinates : {i.location_coor}, area : {i.area}, number: {i.number}")

In [ ]:
no_taxis = 200
taxis = make_taxis(no_taxis)

In [ ]:
pickup = input("Enter your location : ")
pickup = pickup + ',kolkata'

drop_loc = input('Enter your destination : ')
drop_loc = drop_loc + ',kolkata'

arr, label = ask_location(pickup)

taxi, taxis_in_area = allocate_taxi(taxis,arr,label)

dist = compute_distance(pickup,drop_loc)
time_pickup = compute_time(pickup, taxi.location_name + ',kolkata')
time_reach = compute_time(pickup,drop_loc)
fare = compute_fare(pickup,drop_loc)

print(f"\nTaxi number :{taxi.number} from {taxi.location_name} will pick you up from your location in {time_pickup} mins")
print(f'Distance to be travelled is : {dist} km')
print('Your bill will be : Rs', round(fare,2))

if (time_reach>60):
    t_m = time_reach%60
    t_h = int(time_reach/60)
    print(f'You will reach your destination {t_h} hours and {t_m} minutes after departure')
else:
    print(f'You will reach your destination {time_reach} minutes after departure.')
